In [1]:
from Datasets import BinChromaDataset
import numpy as np
from torch.utils.data import DataLoader, Subset
import sys
sys.path.insert(0, '..')
from transformer.models import EncoderOnlyWrapper, EncoderModel
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
import torch
from tqdm import tqdm
import os

In [2]:
npz_path = '../data/augmented_and_padded_data.npz'
dataset = BinChromaDataset(npz_path)

train_percentage = 0.9
split_idx = int( len(dataset)*train_percentage )

train_set = Subset(dataset, range(0,split_idx))
test_set = Subset(dataset, range(split_idx, len(dataset)))

batch_size = 8
epochs = 1000

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=True)

In [3]:
src_vocab_size = 2**12
tgt_vocab_size = 2**12
d_model = 512
num_heads = 8
num_layers = 8
d_ff = 1024
max_seq_length = 129
dropout = 0.3

dev = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

encoderModel = EncoderModel(src_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

transformer = EncoderOnlyWrapper(encoderModel)

transformer = transformer.to(dev)

In [ ]:
criterion = CrossEntropyLoss(ignore_index=0)
optimizer = Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

# keep best validation loss for saving
best_val_loss = np.inf
save_dir = '../saved_models/encoderOnly_one_hot/'
encoder_path = save_dir + 'encoderOnly_one_hot.pt'
os.makedirs(save_dir, exist_ok=True)

for epoch in range(epochs):
    train_loss = 0
    running_loss = 0
    samples_num = 0
    running_accuracy = 0
    accuracy = 0
    with tqdm(train_loader, unit='batch') as tepoch:
        tepoch.set_description(f"Epoch {epoch} | trn")
        for melodies, chords in tepoch:
            melodies = melodies.to(dev)
            chords = chords.to(dev)
            optimizer.zero_grad()
            output = transformer(melodies)
            loss = criterion(output.contiguous().view(-1, tgt_vocab_size), chords.contiguous().view(-1))
            loss.backward()
            optimizer.step()
            # update loss
            samples_num += melodies.shape[0]
            running_loss += loss.item()
            train_loss = running_loss/samples_num
            # accuracy
            prediction = output.argmax(dim=2, keepdim=True).squeeze()
            running_accuracy += (prediction == chords).sum().item()/prediction.shape[1]
            accuracy = running_accuracy/samples_num
            tepoch.set_postfix(loss=train_loss, accuracy=accuracy) # tepoch.set_postfix(loss=loss.item(), accuracy=100. * accuracy)
    # validation
    with torch.no_grad():
        val_loss = 0
        running_loss = 0
        samples_num = 0
        running_accuracy = 0
        accuracy = 0
        print('validation...')
        for melodies, chords in test_loader:
            melodies = melodies.to(dev)
            chords = chords.to(dev)
            output = transformer(melodies)
            loss = criterion(output.contiguous().view(-1, tgt_vocab_size), chords.contiguous().view(-1))
            # update loss
            samples_num += melodies.shape[0]
            running_loss += loss.item()
            val_loss = running_loss/samples_num
            # accuracy
            prediction = output.argmax(dim=2, keepdim=True).squeeze()
            running_accuracy += (prediction == chords).sum().item()/prediction.shape[1]
            accuracy = running_accuracy/samples_num
        if best_val_loss > val_loss:
            print('saving!')
            best_val_loss = val_loss
            torch.save(transformer.state_dict(), encoder_path)
        print(f'validation: accuracy={accuracy}, loss={val_loss}')

Epoch 0 | trn: 100%|██████████████████████████████████████████████████████████████████████████████████| 600/600 [02:18<00:00,  4.33batch/s, accuracy=0.0169, loss=0.68]


validation...
saving!
validation: accuracy=0.03129863141207443, loss=0.5990776031594339


Epoch 1 | trn: 100%|█████████████████████████████████████████████████████████████████████████████████| 600/600 [03:13<00:00,  3.10batch/s, accuracy=0.0359, loss=0.587]


validation...
saving!
validation: accuracy=0.04182846837412919, loss=0.5614077016068221


Epoch 2 | trn: 100%|█████████████████████████████████████████████████████████████████████████████████| 600/600 [02:18<00:00,  4.34batch/s, accuracy=0.0438, loss=0.554]


...

Epoch 29 | trn: 100%|█████████████████████████████████████████████████████████████████████████████████| 600/600 [03:02<00:00,  3.29batch/s, accuracy=0.148, loss=0.241]


validation...
validation: accuracy=0.04889683959451403, loss=0.5841495310835275


Epoch 30 | trn: 100%|█████████████████████████████████████████████████████████████████████████████████| 600/600 [02:18<00:00,  4.35batch/s, accuracy=0.152, loss=0.232]


validation...
validation: accuracy=0.04902773535785447, loss=0.5927000958297758


Epoch 31 | trn:  23%|███████████████████▏                                                              | 140/600 [00:32<01:45,  4.35batch/s, accuracy=0.16, loss=0.217]